In [1]:
import sys
import os
import setuptools.dist
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np
from matplotlib import pyplot as plt
sys.path.append('/home/gt/gdrive/HMSC/2022.06.03_HPC_development/hmsc-hpc')

path_data = "/home/gt/DATA/geolifeclef-2025"
modelTypeString = "cov"

samN = 5
thinN = 10000
nChains = 1
eagerExecFlag = 0
fp = 64
RS = 1
transient = samN*thinN

2025-05-06 06:49:02.593648: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746503342.605920  459870 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746503342.609541  459870 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-06 06:49:02.623259: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
verbose = np.maximum(1, int((transient+samN*thinN)/100))
dtype = np.float32 if fp == 32 else np.float64
tf.config.run_functions_eagerly(eagerExecFlag)
input_path = os.path.join(path_data, "hmsc", "init", "init_%s_chain%.2d.rds" % (modelTypeString, nChains))
output_path = os.path.join(path_data, "hmsc", f"fmTF_gtb{fp}", "TF_%s_chain%.2d_sam%.4d_thin%.4d.rds" % (modelTypeString, nChains, samN, thinN))
os.makedirs(os.path.join(path_data, "hmsc", f"fmTF_gtb{fp}"), exist_ok=True)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=11400)])

# gpus = tf.config.list_physical_devices('GPU')
# tf.config.experimental.set_memory_growth(gpus[0], True)

args="""--samples %d --transient %d --thin %d --verbose %d --input %s --output %s --fse 1 --tnlib tf --profile 0 --fp %d""" % (samN, transient, thinN, verbose, input_path, output_path, fp)
run_cmd = '"/home/gt/gdrive/HMSC/2022.06.03_HPC_development/hmsc-hpc/hmsc/run_gibbs_sampler.py"' + " " + args
GTB%run $run_cmd

args=Namespace(samples=5, transient=50000, thin=10000, chains=None, input='/home/gt/DATA/geolifeclef-2025/hmsc/init/init_cov_chain01.rds', output='/home/gt/DATA/geolifeclef-2025/hmsc/fmTF_gtb64/TF_cov_chain01_sam0005_thin10000.rds', verbose=1000, hmcleapfrog=10, hmcthin=0, updbe=0, tnlib='tf', fse=1, profile=0, rngseed=0, fp=64)
working directory /home/gt/gdrive/geolifeclef/hmsc


I0000 00:00:1746503356.012027  459870 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 11400 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4070 SUPER, pci bus id: 0000:01:00.0, compute capability: 8.9
I0000 00:00:1746503356.164158  459870 cuda_solvers.cc:178] Creating GpuSolver handles for stream 0xaa06da70


Initializing TF graph
retracing
sampling
TF graph initialized in 266.6 sec
Running TF Gibbs sampler for 1 chains with indices [0]

 Computing chain 0
sampling
iteration 100000 / 100000 saving 5
 1 chains completed in 14164.5 sec

 Whole Gibbs sampler elapsed 14164.5
done


### GTB

ns 1000, 100000 iterations, 14164s

### Mahti

ns 1000, 2000 iterations, 32.5s

ns 2519, 2000 iterations, 74s; 200000 iterations, 7440s

### Spatial 100, Mahti

ns 2519, 2000 iterations, 135s

In [ ]:
rsync -av -e ssh gtb:/home/gt/DATA/geolifeclef-2025/hmsc /Users/gtikhono/DATA/geolifeclef/geolifeclef-2025/ ; rsync -av /Users/gtikhono/DATA/geolifeclef/geolifeclef-2025/hmsc/init -e ssh mahti:/scratch/project_2014077/geolifeclef/hmsc